In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [6]:
!pip install llama_index.embeddings.huggingface
!pip install llama_index


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-base-en-v1.5"
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [9]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

month_mapping = {
    "01": "January",
    "02": "February",
    "03": "March",
    "04": "April",
    "05": "May",
    "06": "June",
    "07": "July",
    "08": "August",
    "09": "September",
    "10": "October",
    "11": "November",
    "12": "December"
}
# Define a function to extract metadata
def extract_month_metadata(file_path):
    # Extract month from file name or content
    month = file_path.split('/')[-1].split('.')[0]  # Assuming file name format like 'statement_01.pdf'
    month_name = month_mapping.get(month, "Invalid month")
    print(month_name)
    return {"month": month_name}

# Load documents with metadata
reader = SimpleDirectoryReader(input_dir="../data/output/", file_metadata=extract_month_metadata)
documents = reader.load_data()

# Create an index with the documents
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist(persist_dir="../indexes")

January
February
March
April
May
June
July
August
September
October
November
December


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(model="llama3.2:3b",
                      temperature=0.1,
                      system_prompt="You are a financial assistant, you help with analyzing the bank transactions\
                      . If you are not sure about transaction detail, then you do not give any output and you should say \"I cannot aprovide that information\"",
                      request_timeout=300.0)

In [11]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
    verbose=True,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="compact_accumulate",
    verbose=True,
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# Query the LLM and get response

In [14]:
response = query_engine.query("Can you total debit and credit of transactions for month of March?")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
Response 1: To calculate the total debit and credit for the month of March, we need to sum up the individual debit and credit amounts.

Debit amount for March:
- UPI/305854390318/202476499: 1,22,400.00
- UPI/DR/307847352585/ICCL: 5,000.00
- UPI/DR/308153794815/ICCL: 45,000.00
- UPI/DR/310720952189/IndianC: 20,000.00
- UPI/DR/310720954808/IndianC: 5,000.00

Total debit amount for March = 1,22,400 + 5,000 + 45,000 + 20,000 + 5,000 = 2,97,400

Credit amount for March:
- NEFT*HDFC0000240*N060232: 3,42,366.84
- TRANSFER TO (GAPFSJZI5): 45,873.76
- DEBIT-CREDIT balance (initial): 3,42,366.84

Total credit amount for March = 3,42,366.84 + 45,873.76 = 3,88,240.60
